In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys

#Select Dataset

In [ ]:
#dataset_name = "Multi"
dataset_name = "ARAS"

# Reading in the Data and processing

In [ ]:
if dataset_name == "ARAS":
  df_clean = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/processed_data/Aras_HouseA.csv")
elif dataset_name == "Multi":
  df_clean = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/processed_data/3_months_data.csv")

In [ ]:
df_clean

,Unnamed: 0,sensor_id,value,time,old_sen
0,0,5889,1,49,3
1,1,5889,0,252,3
2,2,5889,1,301,3
3,3,5889,0,504,3
4,4,5889,1,505,3
...,...,...,...,...,...
53873,53873,7125,0,85987,17
53874,53874,5888,0,85992,11
53875,53875,5893,1,86001,15
53876,53876,5893,0,86004,15


# Convertion to format for the SPEED algorithm

In [ ]:
def create_on_off_Multi_char():
    dic_on_off = {}
    dic_on_off[5895] = ["a","A"]
    dic_on_off[7125] = ["b", "B"]
    dic_on_off[5896] = ["c", "C"]
    dic_on_off[6253] = ["d", "D"]
    dic_on_off[6632] = ["e", "E"]
    dic_on_off[6633] = ["f", "F"]
    dic_on_off[6635] = ["g", "G"]
    dic_on_off[6896] = ["h", "H"]
    dic_on_off[5887] = ["i", "I"]
    dic_on_off[5888] = ["j", "J"]
    dic_on_off[5889] = ["k", "K"]
    dic_on_off[5893] = ["l", "L"]
    return dic_on_off

In [ ]:
def create_on_off_Multi_int():
    dic_on_off = {}
    dic_on_off[5895] = [0,1]
    dic_on_off[7125] = [2, 3]
    dic_on_off[5896] = [4,5]
    dic_on_off[6253] = [6, 7]
    dic_on_off[6632] = [8, 9]
    dic_on_off[6633] = [10, 11]
    dic_on_off[6635] = [12, 13]
    dic_on_off[6896] = [14, 15]
    dic_on_off[5887] = [16, 17]
    dic_on_off[5888] = [18, 19]
    dic_on_off[5889] = [20, 21]
    dic_on_off[5893] = [22, 23]
    return dic_on_off

In [ ]:
dic_on_off = create_on_off_Multi_char()
dic_on_off

{5887: ['i', 'I'],
 5888: ['j', 'J'],
 5889: ['k', 'K'],
 5893: ['l', 'L'],
 5895: ['a', 'A'],
 5896: ['c', 'C'],
 6253: ['d', 'D'],
 6632: ['e', 'E'],
 6633: ['f', 'F'],
 6635: ['g', 'G'],
 6896: ['h', 'H'],
 7125: ['b', 'B']}

In [ ]:
dic_on_off_int = create_on_off_Multi_int()
dic_on_off_int

{5887: [16, 17],
 5888: [18, 19],
 5889: [20, 21],
 5893: [22, 23],
 5895: [0, 1],
 5896: [4, 5],
 6253: [6, 7],
 6632: [8, 9],
 6633: [10, 11],
 6635: [12, 13],
 6896: [14, 15],
 7125: [2, 3]}

In [ ]:
def add_event_col(df, dic):

    add = []

    for index, row in df.iterrows():

        sensor = row["sensor_id"]
        value = int(row["value"])

        add.append(dic[sensor][value])

    df["event"] = np.array(add)
    return df

In [ ]:
df = add_event_col(df_clean, dic_on_off)
if dataset_name == "ARAS":
  df = df[["time", "event"]]
elif dataset_name == "Multi":
  df = df[["time_scal", "event"]]

In [ ]:
df["event"].value_counts()

l    9304
L    9264
K    7000
k    6989
B    4855
b    4852
C    3221
c    3216
A    1515
a    1510
d     741
D     740
j     336
J     335
Name: event, dtype: int64

#The SPEED Algorithm

# df and sequence list

In [ ]:
df

,time,event
0,49,K
1,252,k
2,301,K
3,504,k
4,505,K
...,...,...
53873,85987,b
53874,85992,j
53875,86001,L
53876,86004,l


In [ ]:
sequence_list = df.values.tolist()    # [timestamp, single event]
print("length of total sequence:", len(sequence_list))
print("length of training sequence:", int(len(sequence_list)*0.8))
print("length of testing sequence:", int(len(sequence_list)*0.2))

print(sequence_list)

length of total sequence: 53878
length of training sequence: 43102
length of testing sequence: 10775
[[49, 'K'], [252, 'k'], [301, 'K'], [504, 'k'], [505, 'K'], [506, 'k'], [509, 'K'], [510, 'k'], [513, 'K'], [514, 'k'], [515, 'K'], [518, 'k'], [519, 'K'], [530, 'k'], [558, 'L'], [560, 'l'], [592, 'L'], [594, 'l'], [616, 'C'], [797, 'c'], [811, 'L'], [812, 'l'], [822, 'L'], [824, 'l'], [828, 'L'], [831, 'l'], [884, 'K'], [1056, 'k'], [1059, 'K'], [1060, 'k'], [1061, 'K'], [1063, 'k'], [1064, 'K'], [1066, 'k'], [1070, 'K'], [1075, 'k'], [1076, 'K'], [1100, 'k'], [1101, 'K'], [1103, 'k'], [1105, 'K'], [1139, 'k'], [1143, 'K'], [1145, 'k'], [1147, 'K'], [1333, 'k'], [1337, 'K'], [1338, 'k'], [1342, 'K'], [1343, 'k'], [1344, 'K'], [1350, 'k'], [1351, 'K'], [1353, 'k'], [1354, 'K'], [1389, 'k'], [1390, 'K'], [1470, 'k'], [1471, 'K'], [1806, 'k'], [1807, 'K'], [1854, 'k'], [1885, 'K'], [1895, 'k'], [1924, 'K'], [2040, 'k'], [2096, 'K'], [2098, 'k'], [2164, 'K'], [2180, 'k'], [2273, 'K'], [23

In [ ]:
training_data = sequence_list[:int(len(sequence_list)*0.8)]
testing_data = sequence_list[int(len(sequence_list)*0.8):]
print("training:", len(training_data))
print("testing:", len(testing_data))

training: 43102
testing: 10776
bkKaKkBbKkKBbBbBbBbBbBbBbLlLlkKkKkKkKkKkKkKBbLlLlBbBbkLlLlLlLlDdLlLlLlBbBbBbBbLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlDdDdDdLlBbLlLlLlLlLlBbLlkKBbkKkKkKlkKkKkKkKkKkKkKkLlLlLlLlLlLBblLlLlLlLlLlLDLlLlLlLlLlLlLlLlLlLlLlLdlLlDlLdDdlLlKkKkKkKKkKkLlLlKkkKBbkKkKkKkKkKkKkKkKkKkKkLlLlKkKkLlKkKkKkKkKkKkKkKLlkKkKkKkKkKkKkKkKBbBbLlLlLlLlKBkKBbkLlLlkLlLlCcCcCABaAbBbBbacCLlLlLlLlLlLlLlcLlLlLlLlLlLlLlLlLlLlCcCcCcCcCLlLlcCcCcCcCcCcCcCcCcCKDdDdLlLlLlABkaAaALlLlKbBbBbBbaLlkKLlLlLlCkKkKcAkKkKBbkKkKkKkKkKkKkKkKkKkKkKkKkKBbakKKCcCDkcCdLlLlLlLlLlkLlLlLlLcCcCclLlDdLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlDLdlLlLlLlLlLlLlLlLlLlLlLlLlLDldLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlKlLlLlLlLlkKkABbBbBbaLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLKlDdLlkKkKkKLlLlLlLlkKkKkKBbkKBbBbkcBbBbLlLlLlLlLlKkKkKkKkKkKkKCcCcCcCcCcCcCcCcCcCcCcCcCcCcCcCcCcCcABbBbaCcCcCcCkKkKkKkKkKkKkKkKkKkKkKcCcCcCcCcCcCcCcCcABbkKBbaABbBbBbBbBbBbaBbBbBbBbBbBbBbBbBbDdDdLlCcCcCcCcCcCcJjkKkABbBbBbBbBbBbaLlLlKkKkKkKkKkLlLlKkABbBaAaAa

#Create Episodes

In [ ]:
caps = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L"]   # turning on
lows = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l"]   # turning off
all_actions = caps + lows

In [ ]:
def create_episodes(episode_seq, episode_list, sequence_list):
  for item_id, item in enumerate(sequence_list):
    item_time = item[0]
    item_event = item[1]
    episode_seq += item_event
    if item_event in caps:      # start of episode
      episode = item_event
      #print("start of episode: ", item_event)
      opp_event = False
      plus_id = 1
      while opp_event == False and item_id+plus_id < len(sequence_list)-1:
        next_item = sequence_list[item_id + plus_id]
        if next_item[1] == item_event.lower():        # end of episode
          episode += next_item[1]
          episode_list.append(episode)
          opp_event = True
        else:
          episode += next_item[1]
          plus_id += 1
    if item_event in lows:      # start of episode
      episode = item_event
      #print("start of episode: ", item_event)
      opp_event = False
      plus_id = 1
      while opp_event == False and item_id+plus_id < len(sequence_list)-1:
        next_item = sequence_list[item_id + plus_id]
        if next_item[1] == item_event.upper():        # end of episode
          episode += next_item[1]
          episode_list.append(episode)
          opp_event = True
        else:
          episode += next_item[1]
          plus_id += 1
      #print("end of episode, episode: ", episode)

  return episode_list, episode_seq

In [ ]:
# tree[key] = [occurence of key, {"seq1 following key":occur of seq1, "seq2 following key":occur of seq2}]
# episode_seq = sub_episode_seq   # to be deleted later after testing

def create_dict_for_tree(episode_list):
  for ep_id, ep in enumerate(episode_list):
    #print("ep[0]: ", ep[0])
    if ep[0] not in dict_tree.keys():
      dict_tree[ep[0]] = [1, {}]
    else:
      dict_tree[ep[0]][0] += 1
      cumu_seq = ""
      for next_item in ep[1:]:
        cumu_seq += next_item
        if cumu_seq in dict_tree[ep[0]][1].keys():
          dict_tree[ep[0]][1][cumu_seq] += 1
        else:
          dict_tree[ep[0]][1][cumu_seq] = 1

  return dict_tree


In [ ]:
episode_seq = ""
episode_list = list()
dict_tree = dict()     # [occurences of the alphabet, [[list of things that follows, occurences]]]
episode_list, episode_seq = create_episodes(episode_seq, episode_list, training_data)
dict_tree = create_dict_for_tree(episode_list)

In [ ]:
episode_list

['Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kLlLlCcLlLlLlK',
 'Ll',
 'lL',
 'Ll',
 'lCcL',
 'Cc',
 'cLlLlLlKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKABbBbBbBbakKkLlLlLlLlBbBbABbBbBbBbaBbLlC',
 'Ll',
 'lL',
 'Ll',
 'lL',
 'Ll',
 'lKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKkKABbBbBbBbakKkL',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'Kk',
 'kK',
 'KABbBbBbBbak',
 'ABbBbBbBba',
 'Bb',
 'bB',
 'Bb',
 'bB',
 'Bb',
 'bB',
 'Bb',
 'bakKkLlLlLlLlB',
 'akKkLlLlLlLlBbBbA',
 'kK',
 'Kk',
 'kLlLlLlLlBbBbABbBbBbBbaBbLlCLlK',
 'Ll',
 'lL',
 'Ll',
 'lL',
 'Ll',
 'lL',
 'Ll',
 'lBbBbABbBbBbBbaBbL',
 'Bb',
 'bB',
 'Bb',
 'bAB',
 'ABbBbBbBba',
 'Bb',
 'bB',
 'Bb',
 'b

In [ ]:
dict_tree.keys()

dict_keys(['K', 'k', 'L', 'l', 'C', 'c', 'A', 'B', 'b', 'a', 'J', 'j', 'D', 'd'])

In [ ]:
dict_tree

{'A': [1407,
  {'B': 188,
   'Bb': 131,
   'BbB': 98,
   'BbBb': 89,
   'BbBbB': 72,
   'BbBbBb': 70,
   'BbBbBbB': 46,
   'BbBbBbBb': 44,
   'BbBbBbBba': 13,
   'BbBbBba': 20,
   'BbBL': 1,
   'BbBLl': 1,
   'BbBLlb': 1,
   'BbBLlba': 1,
   'L': 11,
   'La': 1,
   'BB': 9,
   'BBb': 7,
   'BBbb': 2,
   'BBbbB': 2,
   'BBbbBb': 2,
   'BBbbBba': 2,
   'a': 1100,
   'Ba': 41,
   'Bba': 20,
   'b': 66,
   'bB': 44,
   'bBb': 37,
   'bBbB': 31,
   'bBbBa': 2,
   'ba': 20,
   'BbBbBbBbB': 31,
   'BbBbBbBbBb': 28,
   'BbBbBbBbBba': 6,
   'Ll': 8,
   'Lla': 2,
   'bBB': 1,
   'bBBb': 1,
   'bBBbB': 1,
   'bBBbBb': 1,
   'bBBbBbb': 1,
   'bBBbBbbB': 1,
   'bBBbBbbBb': 1,
   'bBBbBbbBba': 1,
   'BbBa': 5,
   'BbBbBbBbBbB': 22,
   'BbBbBbBbBbBb': 21,
   'BbBbBbBbBbBbB': 17,
   'BbBbBbBbBbBbBb': 16,
   'BbBbBbBbBbBbBba': 1,
   'BbBba': 14,
   'bBbBb': 29,
   'bBbBba': 13,
   'BbBbBbBbBbBba': 4,
   'bBbBbB': 16,
   'bBbBbBb': 16,
   'bBbBbBba': 8,
   'Bbb': 5,
   'Bbba': 2,
   'BBba': 3,
   'bBba'

#Create nested list as stat tree

In [ ]:
# dictionary created above, used to map to decision tree
for key, item in dict_tree.items():
  print("root: ", key, "\tnumber of times as a root: ", item[0], "\tnumber of children: ", len(item[1]))

root:  K 	number of times as a root:  5616 	number of children:  6662
root:  k 	number of times as a root:  5604 	number of children:  19250
root:  L 	number of times as a root:  7379 	number of children:  362
root:  l 	number of times as a root:  7409 	number of children:  20985
root:  C 	number of times as a root:  2668 	number of children:  1972
root:  c 	number of times as a root:  2658 	number of children:  33341
root:  A 	number of times as a root:  1407 	number of children:  2161
root:  B 	number of times as a root:  3648 	number of children:  1163
root:  b 	number of times as a root:  3640 	number of children:  28913
root:  a 	number of times as a root:  1403 	number of children:  35445
root:  J 	number of times as a root:  246 	number of children:  30
root:  j 	number of times as a root:  246 	number of children:  41850
root:  D 	number of times as a root:  586 	number of children:  277
root:  d 	number of times as a root:  584 	number of children:  33733


In [ ]:
dict_tree['A']

[1407,
 {'B': 188,
  'Bb': 131,
  'BbB': 98,
  'BbBb': 89,
  'BbBbB': 72,
  'BbBbBb': 70,
  'BbBbBbB': 46,
  'BbBbBbBb': 44,
  'BbBbBbBba': 13,
  'BbBbBba': 20,
  'BbBL': 1,
  'BbBLl': 1,
  'BbBLlb': 1,
  'BbBLlba': 1,
  'L': 11,
  'La': 1,
  'BB': 9,
  'BBb': 7,
  'BBbb': 2,
  'BBbbB': 2,
  'BBbbBb': 2,
  'BBbbBba': 2,
  'a': 1100,
  'Ba': 41,
  'Bba': 20,
  'b': 66,
  'bB': 44,
  'bBb': 37,
  'bBbB': 31,
  'bBbBa': 2,
  'ba': 20,
  'BbBbBbBbB': 31,
  'BbBbBbBbBb': 28,
  'BbBbBbBbBba': 6,
  'Ll': 8,
  'Lla': 2,
  'bBB': 1,
  'bBBb': 1,
  'bBBbB': 1,
  'bBBbBb': 1,
  'bBBbBbb': 1,
  'bBBbBbbB': 1,
  'bBBbBbbBb': 1,
  'bBBbBbbBba': 1,
  'BbBa': 5,
  'BbBbBbBbBbB': 22,
  'BbBbBbBbBbBb': 21,
  'BbBbBbBbBbBbB': 17,
  'BbBbBbBbBbBbBb': 16,
  'BbBbBbBbBbBbBba': 1,
  'BbBba': 14,
  'bBbBb': 29,
  'bBbBba': 13,
  'BbBbBbBbBbBba': 4,
  'bBbBbB': 16,
  'bBbBbBb': 16,
  'bBbBbBba': 8,
  'Bbb': 5,
  'Bbba': 2,
  'BBba': 3,
  'bBba': 4,
  'bBa': 5,
  'bBbBbBbB': 8,
  'bBbBbBbBb': 7,
  'bBbBbBbBba':

In [ ]:
def tree_search(tree, parent_seq, node_char, node_count):
  # tree =   [str, int, list] or [str, int]
  check_layer = tree
  for char in parent_seq:
    for item in check_layer:
      #print("now layer: ", check_layer, "item: ", item, "matching char: ", char, "parent seq: ", parent_seq)
      if type(item) == list and item[0] == char:
        if type(item[-1]) == list:
          #print("append to: ", item, "check layer: ", check_layer, "TRIGGERED HERE**")   # next layer would be deeper in the tree
          check_layer = check_layer[-1]
        else:
          #print("append to: ", item, "check layer: ", check_layer, "TRIGGERED HERE")     # next layer would be the most shallow subtree layer
          item.append([node_char, node_count])
  return tree

In [ ]:
def count_nodes(node_dict, node):
  if node in caps or node in lows:
    if node in node_dict:
      node_dict[node] += 1
    else:
      node_dict[node] = 1
  return node_dict

#The Tree

In [ ]:
sys.setrecursionlimit(100000)

In [ ]:
the_tree = list()
count_of_all_nodes = dict()
#root_dict = dict()

print("root nodes", "\t[subtrees]")

for key, item in dict_tree.items(): # key: root node (always caps eg. ABC, one of the 12)
  key_list = item[1].keys()
  sub_tree = list()
  checking = []
  check_now = ""
  count_nodes(count_of_all_nodes, key)
  for sub_key in key_list:  # sub_key: the first alphabet following the root (could be upper/lower case)
    if len(checking) == 0 or len(sub_key) == 1:   # adding the first node after the root
        checking.append(sub_key)    # add into the list of existing nodes (for later checking)
        check_now = sub_key
        sub_tree.append([check_now, item[1][sub_key]])    # adds to subtree
        count_nodes(count_of_all_nodes, item[1][sub_key])
    elif sub_key.startswith(checking[-1]):        # match to the last node, if True the next node should be a child of the last one
      append_node_char = sub_key[len(checking[-1]):]
      if len(append_node_char) == 1:
        #print("sub_key: ", sub_key, "checking: ", check_now, "append: ", append_node_char, "occur: ", item[1][sub_key])
        sub_tree = tree_search(sub_tree, check_now, append_node_char, item[1][sub_key])
        count_nodes(count_of_all_nodes, append_node_char)
      check_now = sub_key
      checking.append(check_now)
    elif sub_key.startswith(checking[-1]) == False:
      re_check_id = -2  #check one more index back
      check_now = checking[re_check_id]
      while check_now != checking[0]:
        if sub_key.startswith(checking[re_check_id]):
          append_node_char = sub_key[len(checking[re_check_id]):]
          sub_tree = tree_search(sub_tree, check_now, append_node_char, item[1][sub_key])
          count_nodes(count_of_all_nodes, append_node_char)
          check_now = sub_key
          checking.append(check_now)
          break
        else:
          #print("*RECHECK*", "sub_key: ", sub_key, "checking: ", check_now, "append: ", "nothing")
          re_check_id -= 1
          check_now = checking[re_check_id]
  the_tree.append([key, count_of_all_nodes[key], sub_tree])
  #root_dict[key] = sub_tree

  print(key, "\t", sub_tree)

print("count of all nodes")
count_of_all_nodes

root nodes 	[subtrees]
K 	 [['k', 5026], ['A', 27, ['B', 6, ['b', 5, ['B', 4, ['b', 4, ['B', 4, ['b', 4, ['B', 3, ['b', 3, ['a', 2, ['k', 2]]]]]]]]]]], ['L', 110, ['l', 94, ['L', 68, ['l', 63, ['L', 43, ['l', 38, ['L', 25, ['l', 25, ['c', 1, ['B', 1, ['b', 1, ['B', 1, ['b', 1, ['L', 1, ['l', 1, ['B', 1, ['b', 1, ['L', 1, ['l', 1, ['L', 1, ['l', 1, ['B', 1, ['b', 1, ['B', 1, ['b', 1, ['B', 1, ['b', 1, ['B', 1, ['b', 1, ['B', 1, ['b', 1, ['B', 1, ['b', 1, ['B', 1, ['b', 1, ['B', 1, ['b', 1, ['B', 1, ['b', 1, ['B', 1, ['b', 1, ['A', 1, ['B', 1, ['b', 1, ['B', 1, ['b', 1, ['B', 1, ['b', 1, ['a', 1, ['C', 1, ['c', 1, ['J', 1, ['j', 1, ['k', 1]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]], ['J', 10, ['j', 7, ['J', 4, ['j', 4, ['k', 2]]]]], ['d', 9, ['L', 3, ['l', 2, ['L', 1, ['l', 1, ['L', 1, ['l', 1, ['L', 1, ['l', 1, ['L', 1, ['l', 1, ['L', 1, ['l', 1, ['L', 1, ['l', 1, ['L', 1, ['l', 1, ['L', 1, ['l', 1, ['L', 1, ['l', 1, ['k', 1]]]]]]]]]]]]]]]]]]]]]], ['l', 64, ['L', 36, ['l', 26

{'A': 7376,
 'B': 18462,
 'C': 13968,
 'D': 3194,
 'J': 1343,
 'K': 29896,
 'L': 37601,
 'a': 7277,
 'b': 18093,
 'c': 13853,
 'd': 2980,
 'j': 1254,
 'k': 29771,
 'l': 37303}

In [ ]:
total_count_of_nodes = 0
for key, item in count_of_all_nodes.items():
  total_count_of_nodes += item

print(total_count_of_nodes)
print(len(count_of_all_nodes))

222371
14


In [ ]:
the_tree[1]

['k',
 316,
 [['K', 4986],
  ['L',
   238,
   ['l',
    224,
    ['L',
     167,
     ['l',
      160,
      ['C',
       3,
       ['c',
        3,
        ['L',
         1,
         ['l', 1, ['L', 1, ['l', 1, ['L', 1, ['l', 1, ['K', 1]]]]]]]]]]]]],
  ['B',
   77,
   ['b',
    70,
    ['B',
     37,
     ['b',
      36,
      ['L', 6, ['l', 6, ['L', 4, ['l', 4, ['L', 3, ['l', 3, ['K', 1]]]]]]]]]]],
  ['D',
   30,
   ['L',
    3,
    ['l',
     3,
     ['d',
      2,
      ['L',
       2,
       ['l',
        2,
        ['L',
         2,
         ['l',
          2,
          ['L',
           2,
           ['l',
            2,
            ['L',
             2,
             ['l',
              2,
              ['L',
               2,
               ['l',
                2,
                ['L',
                 2,
                 ['l',
                  2,
                  ['L',
                   2,
                   ['l',
                    2,
                    ['L',
            

In [ ]:
len(the_tree)

14

#Prediction with reference to tree

modify below

In [ ]:
def get_store_list(target_count_in_term_node, check_seq, target, the_tree, check_subseq, check_layer, store_for_prob, counter):
        for id, thing in enumerate(check_layer):
          if counter == 0:
            if thing == target:
              #print("target: ", thing, check_layer[id+1])
              target_count_in_term_node = check_layer[id+1]
              break
          
          if type(thing) == str and len(check_subseq) == 1:
            #print(thing, check_subseq)
            if thing == check_subseq[0]:
              #print("found match: ", thing, "in node", check_layer)
              # store numbers
              store_for_prob.append((thing, check_layer[1]))
              #print("append:", (thing, check_layer[1]))
          elif type(thing) == int:
            continue
          elif type(thing[-1]) == list and len(check_subseq)>0:
            if thing[0] == check_subseq[0]:
              check_layer = thing[2]
              #print("found match: ", thing)
              # store nums
              store_for_prob.append((thing[0],thing[1]))
              #print("append:", (thing[0],thing[1]))
        return store_for_prob, target_count_in_term_node


In [ ]:
def stored_for_prob(target_count_in_term_node, check_seq, target, the_tree):
  store_for_prob = list()
  for root in the_tree:
    if root[0] == check_seq[0]:
      #print(root[0], root)
      root_subtree = root
      #print("root_subtree:", root)
      #print("branches to", root[0], ":", len(root[2]))

      check_subseq = check_seq[1:]
      counter = len(check_subseq)
      check_layer = root[2]

      while counter > -2:
        for sub_item in check_layer:
          #print("counter: ", counter, "check_subseq: ", check_subseq, "check layer: ", sub_item)
          store_for_prob, target_count_in_term_node = get_store_list(target_count_in_term_node, check_seq, target, the_tree, check_subseq, sub_item, store_for_prob, counter)
          check_subseq = check_subseq[1:]
        counter -= 1
  return(store_for_prob, target_count_in_term_node, root_subtree)

In [ ]:
def prob(root, target, check_seq, store_for_prob, target_count_in_term_node, root_subtree):
  target_in_all_nodes = count_of_all_nodes[target]/total_count_of_nodes
  prob = (target_count_in_term_node/root_subtree[1])*target_in_all_nodes
  prob_output = prob

  for item in store_for_prob:
    new_prob = (target_count_in_term_node/item[1])*prob_output
    #print(item, "update:", target_count_in_term_node, "/", item[1], "*", prob_output)
    prob_output = new_prob
    #print("prob:" , prob_output)

  return prob_output

In [ ]:
def prob_of_target(check_seq, target, the_tree, total_count_of_nodes):
  root = check_seq[0]
  root_subtree = list()
  store_for_prob = list()
  target_count_in_term_node = 0

  store_for_prob, target_count_in_term_node, root_subtree = stored_for_prob(target_count_in_term_node, check_seq, target, the_tree)

  #print("store_for_prob:", store_for_prob)
  #print("target_count_in_term_node", target_count_in_term_node)
  #print("root_subtree", root_subtree)

  target_in_all_nodes = count_of_all_nodes[target]/total_count_of_nodes
  #print(target_in_all_nodes)
  '''
  print("Given sequence:", check_seq)
  print("Predict probability for:", target)
  print("subtree node counts stored for calculation:", store_for_prob)
  print("total number of nodes in the tree:", total_count_of_nodes)
  print("count of target node following the subtree:", target_count_in_term_node)
  print("count of target in the entire tree:", count_of_all_nodes[target])
  print(root_subtree[0], root_subtree[1], len(root_subtree[2]))
  '''

  prob_output = prob(root, target, check_seq, store_for_prob, target_count_in_term_node, root_subtree)

  return prob_output

In [ ]:
# for single instance
check_seq = "Li"
target = "D"
the_prob = prob_of_target(check_seq, target, the_tree, total_count_of_nodes)
print(the_prob)

5.795581861978376e-05


In [ ]:
# given history, prob of all next actions, and prediction of most likely one
check_seq = "B"
predict_prob = list()

for act in all_actions:
  target = act
  if target in count_of_all_nodes.keys():
    the_prob = prob_of_target(check_seq, target, the_tree, total_count_of_nodes)
    predict_prob.append((act, the_prob))
  else:
    predict_prob.append((act, 0))

#print(predict_prob)

prediction = max(predict_prob, key=lambda x:x[1])
#print(prediction)

print("Given sequence:", check_seq)
print("Predicted next action, probability:", prediction)


Given sequence: B
Predicted next action, probability: ('b', 0.03210672153106703)


In [ ]:
prediction = max(predict_prob, key=lambda x:x[1])
print(prediction)

('b', 0.03210672153106703)


# Looping through the dataset for accuracy scores

In [ ]:
# for reference
'''
training_data = sequence_list[:int(len(sequence_list)*0.8)]
testing_data = sequence_list[int(len(sequence_list)*0.8):]
print("training:", len(training_data))
print("testing:", len(testing_data))
print("number of episodes:", len(episode_list))
'''
testing_sequence = ""

for item in testing_data:
  testing_sequence += item[1]


print(testing_sequence)

bkKaKkBbKkKBbBbBbBbBbBbBbLlLlkKkKkKkKkKkKkKBbLlLlBbBbkLlLlLlLlDdLlLlLlBbBbBbBbLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlDdDdDdLlBbLlLlLlLlLlBbLlkKBbkKkKkKlkKkKkKkKkKkKkKkLlLlLlLlLlLBblLlLlLlLlLlLDLlLlLlLlLlLlLlLlLlLlLlLdlLlDlLdDdlLlKkKkKkKKkKkLlLlKkkKBbkKkKkKkKkKkKkKkKkKkKkLlLlKkKkLlKkKkKkKkKkKkKkKLlkKkKkKkKkKkKkKkKBbBbLlLlLlLlKBkKBbkLlLlkLlLlCcCcCABaAbBbBbacCLlLlLlLlLlLlLlcLlLlLlLlLlLlLlLlLlLlCcCcCcCcCLlLlcCcCcCcCcCcCcCcCcCKDdDdLlLlLlABkaAaALlLlKbBbBbBbaLlkKLlLlLlCkKkKcAkKkKBbkKkKkKkKkKkKkKkKkKkKkKkKkKBbakKKCcCDkcCdLlLlLlLlLlkLlLlLlLcCcCclLlDdLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlDLdlLlLlLlLlLlLlLlLlLlLlLlLlLDldLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlKlLlLlLlLlkKkABbBbBbaLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLKlDdLlkKkKkKLlLlLlLlkKkKkKBbkKBbBbkcBbBbLlLlLlLlLlKkKkKkKkKkKkKCcCcCcCcCcCcCcCcCcCcCcCcCcCcCcCcCcCcABbBbaCcCcCcCkKkKkKkKkKkKkKkKkKkKkKcCcCcCcCcCcCcCcCcABbkKBbaABbBbBbBbBbBbaBbBbBbBbBbBbBbBbBbDdDdLlCcCcCcCcCcCcJjkKkABbBbBbBbBbBbaLlLlKkKkKkKkKkLlLlKkABbBaAaAaAbBbBbBbBbBbBbBbaDdDdLlLlLlLlLl

In [ ]:
def get_next_act_prediction(check_seq, the_tree, total_count_of_nodes, all_actions):
  check_seq = check_seq
  predict_prob = list()

  for act in all_actions:
    target = act
    if target in count_of_all_nodes.keys():
      the_prob = prob_of_target(check_seq, target, the_tree, total_count_of_nodes)
      predict_prob.append((act, the_prob))
    else:
      predict_prob.append((act, 0))

  prediction = max(predict_prob, key=lambda x:x[1])

  return predict_prob, prediction

In [ ]:
# testing on testing data episodes
'''
print("training:", len(training_data))
print("testing:", len(testing_data))
print("number of episodes:", len(episode_list))
'''

# testing on sequence
testing_sequence = ""
for item in testing_data:
  testing_sequence += item[1]
print(testing_sequence)

# testing on episodes
test_episode_seq = ""
test_episode_list = list()
test_episode_list, test_episode_seq = create_episodes(episode_seq, episode_list, testing_data)

bkKaKkBbKkKBbBbBbBbBbBbBbLlLlkKkKkKkKkKkKkKBbLlLlBbBbkLlLlLlLlDdLlLlLlBbBbBbBbLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlDdDdDdLlBbLlLlLlLlLlBbLlkKBbkKkKkKlkKkKkKkKkKkKkKkLlLlLlLlLlLBblLlLlLlLlLlLDLlLlLlLlLlLlLlLlLlLlLlLdlLlDlLdDdlLlKkKkKkKKkKkLlLlKkkKBbkKkKkKkKkKkKkKkKkKkKkLlLlKkKkLlKkKkKkKkKkKkKkKLlkKkKkKkKkKkKkKkKBbBbLlLlLlLlKBkKBbkLlLlkLlLlCcCcCABaAbBbBbacCLlLlLlLlLlLlLlcLlLlLlLlLlLlLlLlLlLlCcCcCcCcCLlLlcCcCcCcCcCcCcCcCcCKDdDdLlLlLlABkaAaALlLlKbBbBbBbaLlkKLlLlLlCkKkKcAkKkKBbkKkKkKkKkKkKkKkKkKkKkKkKkKBbakKKCcCDkcCdLlLlLlLlLlkLlLlLlLcCcCclLlDdLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlDLdlLlLlLlLlLlLlLlLlLlLlLlLlLDldLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlKlLlLlLlLlkKkABbBbBbaLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLlLKlDdLlkKkKkKLlLlLlLlkKkKkKBbkKBbBbkcBbBbLlLlLlLlLlKkKkKkKkKkKkKCcCcCcCcCcCcCcCcCcCcCcCcCcCcCcCcCcCcABbBbaCcCcCcCkKkKkKkKkKkKkKkKkKkKkKcCcCcCcCcCcCcCcCcABbkKBbaABbBbBbBbBbBbaBbBbBbBbBbBbBbBbBbDdDdLlCcCcCcCcCcCcJjkKkABbBbBbBbBbBbaLlLlKkKkKkKkKkLlLlKkABbBaAaAaAbBbBbBbBbBbBbBbaDdDdLlLlLlLlLl

In [ ]:
check_seq = "L"
predict_prob, prediction = get_next_act_prediction(check_seq, the_tree, total_count_of_nodes, all_actions)

print("Given sequence:", check_seq)
print("Probability of next action:\n", predict_prob)
print("Prediction of next action:\n", prediction)

Given sequence: L
Probability of next action:
 [('A', 3.345977756258023e-05), ('B', 0.0007956178046262722), ('C', 0.0001056051547799441), ('D', 5.795581861978376e-05), ('E', 0), ('F', 0), ('G', 0), ('H', 0), ('I', 0), ('J', 1.0153760228340845e-06), ('K', 0.0010397329505717038), ('L', 0.0), ('a', 2.2007122317687664e-05), ('b', 0.0001367922440888838), ('c', 4.1894278613017344e-05), ('d', 6.759092810228381e-05), ('e', 0), ('f', 0), ('g', 0), ('h', 0), ('i', 0), ('j', 9.480875149917661e-07), ('k', 0.0010128772754361198), ('l', 0.20136911579123465)]
Prediction of next action:
 ('l', 0.20136911579123465)


In [ ]:
# example of moving windows
# *******not actual predictions
window_size = [3,4,5,6,7,8,9,10]
total_pred = 0
correct_pred = 0

for ws in window_size:
  total_pred = 0
  correct_pred = 0

  print("In window size", ws, "\t For example, given:", testing_sequence[0:0+ws], "... predict", testing_sequence[0+ws+1])


In window size 1 	 For example, given: b ... predict K
In window size 2 	 For example, given: bk ... predict a
In window size 3 	 For example, given: bkK ... predict K
In window size 4 	 For example, given: bkKa ... predict k
In window size 5 	 For example, given: bkKaK ... predict B
In window size 6 	 For example, given: bkKaKk ... predict b
In window size 7 	 For example, given: bkKaKkB ... predict K
In window size 8 	 For example, given: bkKaKkBb ... predict k
In window size 9 	 For example, given: bkKaKkBbK ... predict K
In window size 10 	 For example, given: bkKaKkBbKk ... predict B
